In [1]:
import os, sys

fileDir = os.getcwd()
path = os.path.join(fileDir, "..")
sys.path.append(path)

from utilities import *
import pyspark.sql.functions as F

In [2]:
session = create_spark_session("Neo4j GitHub", SparkConnector.NEO4J)

Added dependencies: 
 ['neo4j-connector-apache-spark_2.12-5.0.1_for_spark_3.jar']


In [3]:
common_URI = "hdfs://namenode:9000//data-team/"

In [4]:
#files = session.read.json(f"{common_URI}files.json")
#files.printSchema()

In [5]:
languages = session.read.json(f"{common_URI}languages.json")
languages.printSchema()

root
 |-- language: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- bytes: string (nullable = true)
 |    |    |-- name: string (nullable = true)
 |-- repo_name: string (nullable = true)



In [6]:
licences = session.read.json(f"{common_URI}licenses.json")
licences.printSchema()

root
 |-- license: string (nullable = true)
 |-- repo_name: string (nullable = true)



In [7]:
#commits = session.read.json(f"{common_URI}commits.json")
#commits.printSchema()

In [8]:
#git_commits= commits.select("commit", "subject", "message", "committer.date", "author.date")

In [9]:
#newColumns = ["id","title","message","committer_date", "author_date"]
#git_commits=git_commits.toDF(*newColumns)
#git_commits.printSchema()

In [10]:
#repositories = commits.select("repo_name")
#repositories = repositories.withColumn("name",F.explode(commits.repo_name)).select("name")
#repositories.printSchema()

In [11]:
git_languages = languages.withColumn("name",F.explode(languages["language.name"])).select("name")
git_languages.printSchema()

root
 |-- name: string (nullable = true)



In [12]:
git_licenses = licences.select("license").withColumnRenamed("license","name")
git_licenses.printSchema()

root
 |-- name: string (nullable = true)



In [13]:
#git_files = files.select("id","ref","path","mode","symlink_target")
#git_files.printSchema()

In [14]:
options = get_default_options(SparkConnector.NEO4J)

options["node.keys"] = "name"
options["labels"] = ":GitLanguage"

spark_write(SparkConnector.NEO4J, git_languages, "Overwrite", options=options)

Dataframe saved to NEO4J


In [15]:
options = get_default_options(SparkConnector.NEO4J)

options["node.keys"] = "name"
options["labels"] = ":GitLicense"

spark_write(SparkConnector.NEO4J, git_licenses, "Overwrite", options=options)

Dataframe saved to NEO4J


In [16]:
#options = get_default_options(SparkConnector.NEO4J)

#options["node.keys"] = "id"
#options["labels"] = ":GitFile"

#spark_write(SparkConnector.NEO4J, git_files, "Overwrite", options=options)

In [17]:
#options = get_default_options(SparkConnector.NEO4J)

#options["node.keys"] = "id"
#options["labels"] = ":GitCommit"

#spark_write(SparkConnector.NEO4J, git_commits, "Overwrite", options=options)